# 데이터 불러오기

In [1]:
import numpy as np
import pandas as pd
from konlpy.tag import Okt

# 데이터 불러오기

In [2]:
# 핵심 문장
keysentence = pd.read_csv('data/keysentence.csv',index_col='Unnamed: 0')
keysentence

,keysentence
0,여성 페미니스트의 주장 20대 남자는 매우 미숙하니 투표권 없애야 하고 만 16세 ...
1,우리나라는 진보나 보수나 진정 국민을 생각하는 정치인들은 없음.
2,"10대 남성이 갈아엎을 거 기다리지 말고 지금 20, 30대가 갈아엎어야지 뭘 그때..."
3,저 지지율이 정말 경악스럽네요. 아직도 지지하는 사람들이 있는 게 참 내가 하면 로...
4,"이게 시작이야.. 서울시장, 대선, 그리고 국회의원 선거로 차례차례 무능하면서 착한..."
...,...
995,3.6 퍼도 아니고 진짜 36% 맞나 조사업체 못 믿겠네….
996,대한민국이 등을 돌렸다. 이제 저들에겐 전자 개표기뿐이다.
997,방역을 잘해서 백신도 못 구해서 쩔쩔매고 앉았나 ㅋㅋ
998,이인자 삼인자에 머물러야 할 놈이 일인자가 됐으니 개판이지 뭐


In [3]:
# 후보 공약
pledge = pd.read_csv('data/pledge.csv',index_col='Unnamed: 0')
pledge

,name,pledge
0,박영선,"21분 생활권 도시 서울, 세계 디지털 경제수도 서울 구현"
1,박영선,"21분안에 주거, 직장, 쇼핑., 여가, 건강, 의료, 교육, 보육이 해결되는 도시"
2,박영선,강남 강북 균형발전 추진
3,박영선,대전환펀드 1조원 구성
4,박영선,"블록체인 기술 기반 KS코인 운용. Al,5G 등 철단기술이 융합된 경제도시 조성"
...,...,...
92,박형준,저출산 예산 1조원대 증액
93,박형준,"정자,난자 보관은행 운영사업추진"
94,박형준,영유마 돌봄 확대사업 추진
95,박형준,팸테크밸리 조성


# 후보공약과 핵심 문장 간의 유사도 파악

## n-gram 사용

In [4]:
# n-gram 어휘 벡터 생성
def ngram(s, num):
    res = []
    slen = len(s)-num+1
    for i in range(slen):
        ss = s[i:i+num]
        res.append(ss)
    return res

# 유사도 분석(빈도를 활용해서)
def diff_ngram(sa, sb, num):
    a = ngram(sa, num)
    b = ngram(sb, num)
    r = []
    cnt = 0
    for i in a:
        for j in b:
            if i==j:
                cnt += 1
                r.append(i)
    return cnt/len(a), r

a = '오늘 강남에서 맛있는 스파게티를 먹었다.'
b = '강남에서 먹었던 오늘의 스파게티는 맛있었다.'

# 2-gram
r2, word2 = diff_ngram(a, b, 2)
print("2-gram:", r2, word2)

# 3-gram
r3, word3 = diff_ngram(a, b, 3)
print("3-gram:", r3, word3)

2-gram: 0.7619047619047619 ['오늘', '강남', '남에', '에서', '서 ', ' 맛', '맛있', '는 ', ' 스', '스파', '파게', '게티', ' 먹', '먹었', '었다', '다.']
3-gram: 0.45 ['강남에', '남에서', '에서 ', ' 맛있', ' 스파', '스파게', '파게티', ' 먹었', '었다.']


- n-gram으로 문장 나누는 것이 정확하지 않은 듯 해서 다른 방법 시도

## TF-IDF로 Vectorization한 후 유사도(similarity) 측정

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

doc_list = pledge['pledge'].tolist()

tfidf_vectorizer = TfidfVectorizer(min_df=1)
tfidf_matrix = tfidf_vectorizer.fit_transform(doc_list)
doc_similarities = (tfidf_matrix * tfidf_matrix.T)

print(doc_list)
print(doc_similarities.toarray())

['21분 생활권 도시 서울, 세계 디지털 경제수도 서울 구현', '21분안에 주거, 직장, 쇼핑., 여가, 건강, 의료, 교육, 보육이 해결되는 도시', '강남 강북 균형발전 추진', '대전환펀드 1조원 구성', '블록체인 기술 기반 KS코인 운용. Al,5G 등 철단기술이 융합된 경제도시 조성', '집 격정 없는 서울, 클러스터 중심 일자리창출과 서울형 창업생태계 조성', '평당 천만원 반값아파트 공공주택 30만호 공급', '청년 등 전월세 보증금 무이자 지원, 최저주거기준 주택 개선자금 지원', '소상공인 임대료 30% 감면, 임대업자 15% 시비 지원', '특수고용직, 플랫폼노동자, 프리랜서 등 고용보험료 지원', '기후와 환경, 교통 대전환', '21분 녹색길 조성', '지하철 1,2,4호선 경의중앙선 지상구간 지하화', '경부고속도로 양재-한남구간, 동서부간선도로 지하화', '2030년부터 내연차 신규등록 금지', '대상별 맞춤형 돌봄과 교육으로 건강하고 행복한 공동체 서울', '유치원 전면 무상급식 추진-어린이집 교사대비 아동비율 개선', '장애인, 어르신을 위한 돌봄SOS센터 인력확충', '원스톱 헬스케어 센터 중심 의료서비스 확대', '재난과 위기에 강한 서울, 세계 문화의 새로운 중심 서울', '문화 패스 카드 도입과 확대, 예술인 고용산재보험 가입지원', '시민주도형 재난안전 협의체 구성등으로 안전관리체계 개편', '동물병원 진료항목 표준화, 진료비 공시제 도입', '주택공급 활성화로 부동산 시장 안정화', '1년내 서울시 도시계획규제 혁파', '재개발, 재건축 정상화로 18.5만호 추진동력 확보', '도심형타운하우스 모아주택 도입으로 3만호 공급', '장기전세주택 시즌2 상생주택으로 7만호 공급', '교통소외지역 연결 및 편의증대', '다리공사 국내 최장기록 달성한 월드컵대교 신속 준공 및 개통', '티스푼공사로 13년째 공사중인 동부간선도로 확장공사 완료', '율곡로 도로구조개선사업 보행로 신속조성추진', '면목선, 난곡선, 목동선, 우

- 자기 자신과, 문장 형태가 아주 비슷한 케이스를 제외하고는 유사도가 그리 높지 않음
- konlpy 패키지를 사용해 특정 품사(명사)만 추출하고 유사도 개선해보자

### konlpy로 특정 품사(명사)만 추출하고 코사인 유사도 측정

In [8]:
okt = Okt()

cnt = [0,0,0,0]
candidate = ['박영선','오세훈','김영춘','박형준']

for i in keysentence['keysentence'].tolist():
    
    doc_list = pledge['pledge'].tolist()
    doc_list.append(i)
    
    doc_nouns_list = [' '.join(okt.nouns(doc)) for doc in doc_list]
    tfidf_vectorizer = TfidfVectorizer(min_df=1)
    tfidf_matrix = tfidf_vectorizer.fit_transform(doc_nouns_list)
    doc_nouns_similarities = (tfidf_matrix * tfidf_matrix.T)
    
    similarity = doc_nouns_similarities.toarray().tolist()
    
    similarity[-1][-1] = 0 # 자신과의 유사도를 0으로 처리(max값 선정할 때 제외하기 위함)
    
    # 유사도 max값이 0.15 이하인 경우 제외
    if max(similarity[-1]) <= 0.15:
        continue
    
    print(f'유사도 벡터')
    print(similarity[-1],'\n')
    print(f'최대 유사도 : {max(similarity[-1])}')
    print(f'핵심 문장 : {i}')
    print(f'최대 유사한 공약 : {pledge.iloc[similarity[-1].index(max(similarity[-1])),:]["pledge"]}')
    print('\n'+'---'*50)
    
    idxmax = similarity[-1].index(max(similarity[-1]))
    
    # 핵심 문장과 가장 비슷한 공약을 선언한 후보 반환하여 카운트
    most_similar_candidate = pledge.iloc[idxmax,:]['name']

    for j in range(len(candidate)):
        if most_similar_candidate == candidate[j]:
            cnt[j] += 1

유사도 벡터
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.19424785527844463, 0.0, 0.0, 0.0, 0.0, 0.0, 0] 

최대 유사도 : 0.19424785527844463
핵심 문장 : 여성 페미니스트의 주장 20대 남자는 매우 미숙하니 투표권 없애야 하고 만 16세 생리 이후 여자는 여성으로 보고 투표권 행세해야 합니다.
최대 유사한 공약 : 고품질 복지로 여성과 아이들이 행복한 부산

------------------------------------------------------------------------------------------------------------------------------------------------------
유사도 벡터
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.07217264173751411, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

유사도 벡터
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2867056359777819, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.3132377002650778, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0] 

최대 유사도 : 0.3132377002650778
핵심 문장 : 집을 못 사니 흥분하지 어떻게 아파트가 투자됐냐?
최대 유사한 공약 : 10만호 노후아파트 리모델링 지원

------------------------------------------------------------------------------------------------------------------------------------------------------
유사도 벡터
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.

유사도 벡터
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.10452495327438338, 0.16485863054252553, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0] 

최대 유사도 : 0.16485863054252553
핵심 문장 : 도로 주차 정비 좀 해주세요. 정말 위험하고 심각합니다.
최대 유사한 공약 : 율곡로 도로구조개선사업 보행로 신속조성추진

------------------------------------------------------------------------------------------------------------------------------------------------------
유사도 벡터
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 

유사도 벡터
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.08867940451380736, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.16947684531318952, 0.0, 0.0, 0.0, 0.0, 0.10593047524885399, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.14329644756924698, 0.22593943270091288, 0.1723606356294565, 0.25345963272251326, 0.20690837765764253, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0] 

최대 유사도 : 0.25345963272251326
핵심 문장 : 일자리도 아주 고급으로 23배 늘어나겠죠.. 잘해보세요
최대 유사한 공약 : 세대맞춤형 특화일자리 창출

------------------------------------------------------------------------------------------------------------------------------------------------------
유사도 벡터
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 

유사도 벡터
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.26622911142136996, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0] 

최대 유사도 : 0.26622911142136996
핵심 문장 : 내 집 마련... 이루지 못할 꿈인 것 같네요
최대 유사한 공약 : 취업교육-취업과 창업-자산형성-내집 마련으로 이어지는 실질적이고 전방위적인 터널탈출 프로젝트

------------------------------------------------------------------------------------------------------------------------------------------------------
유사도 벡터
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.07192553299459857, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0

유사도 벡터
[0.0, 0.0, 0.0, 0.0, 0.0, 0.1726501289573521, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.14835895051717132, 0.0, 0.24911443234803826, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0] 

최대 유사도 : 0.24911443234803826
핵심 문장 : 창업 왜 하냐 ㅋㅋ? 남 밑에서 최저임금 받으면서 일하세요. 제 발 ㅋㅋ
최대 유사한 공약 : 취업,창업 특강 라이브 제공

------------------------------------------------------------------------------------------------------------------------------------------------------
유사도 벡터
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

유사도 벡터
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1877313411446099, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.21365679785518366, 0.16292428092875094, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1499743367528494, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0] 

최대 유사도 : 0.21365679785518366
핵심 문장 : 민희 씨 날이 갈수록 아름다워 지 내 요 행복한 남한 생활 이어가세요
최대 유사한 공약 : 15분도시로 편리한 시민생활 보장

------------------------------------------------------------------------------------------------------------------------------------------------------
유사도 벡터
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0

In [9]:
df = pd.DataFrame()
df['name'] = candidate
df['keysentence_count'] = cnt
df.groupby('name').sum().sort_values(by='keysentence_count',ascending=False)

,keysentence_count
name,
박형준,28
김영춘,20
오세훈,13
박영선,12


- 결과 uhm...
- 개선 방향 생각해보자